In [10]:
import numpy as np
import os 
import skimage.io as io
from PIL import Image
import zipfile
from tqdm import tqdm
import pandas as pd
import threading


In [2]:
import os
if hasattr(os, 'add_dll_directory'):
    with os.add_dll_directory(r'C:\Users\emre\Desktop\Senior\openslide-win64-20230414\bin'):
        import openslide
else:
    import openslide

In [3]:
def check_background(img,upper=225, lower = 20):
    """
    Checks whether the given patch image is a background image or not.
    
    Parameters:
    img (numpy.ndarray): Image to be checked.
    upper (int): Upper threshold value for background. Default value: 225.
    lower (int): Lower threshold value for background. Default value: 20.
    
    Returns:
    bool: Returns True if the image is background, False otherwise.
    """
    if np.mean(img) > upper or np.mean(img) < lower:
        return True
    else:
        return False

In [6]:
def crop_and_save(i,j,folder,tile_size=256):
    """
    Saves a region by cropping at specified coordinates.
    
    Parameters:
    i (int): Starting column coordinate.
    j (int): Starting row coordinate.
    folder (str): Name of the folder to which the cropped region belongs.
    tile_size (int): Size of the cropped region. Default value: 256.
    """
    global wsi
    global mask
    global save_img_path
    global save_mask_path

    wsi_tile = wsi.read_region((i, j), 0, (tile_size, tile_size))
    mask_tile = mask.crop((i, j, i+tile_size, j+tile_size))
    if not (check_background(wsi_tile) or wsi_tile.size != (tile_size,tile_size)):       
        wsi_tile_path = os.path.join(save_img_path,f"{folder}_{i}_{j}_img.png")
        mask_tile_path = os.path.join(save_mask_path,f"{folder}_{i}_{j}_mask.png")
        wsi_tile.save(wsi_tile_path)
        mask_tile.save(mask_tile_path)

In [7]:
def run_batch(i,s,e,folder,tile_size=256):
  threadlist = []
  for j in range(s,e,tile_size):
    t = threading.Thread(target=crop_and_save,args=(i,j,folder,tile_size,))
    threadlist.append(t)
    t.start()

  for tr in threadlist:
    tr.join()

In [ ]:
dir_path = os.path.join(os.getcwd(),'data','files') 
folders = [f for f in os.listdir(dir_path)]
folders.sort()

In [12]:
# Path of the folder containing WSIs and their masks.
dir_path = os.path.join(os.getcwd(),'data','files') 

folders = [f for f in os.listdir(dir_path)]
folders.sort()
# The path to the folder where patches will be saved after the patching process.
patch_path = os.path.join(os.getcwd(),'data','patch') 
tile_size = 256

for folder in tqdm(folders):

  images_path = os.path.join(dir_path,folder)
  wsi_path = [f for f in os.listdir(images_path)  if f.endswith(('svs','SVS'))][0]
  wsi_path = os.path.join(images_path, wsi_path)
  mask_path = [f for f in os.listdir(images_path) if f.endswith('viable.tif')][0]
  mask_path = os.path.join(images_path, mask_path)

  save_img_path = os.path.join(patch_path, folder , folder+'_img')
  save_mask_path = os.path.join(patch_path, folder , folder+'_mask')
  isExist = os.path.exists(save_img_path) and os.path.exists(save_mask_path)

  if not isExist:
    os.makedirs(save_img_path)
    os.makedirs(save_mask_path)
  else:
    continue

  tqdm.write(f"This stage is processing for {folder}")
  wsi = openslide.open_slide(wsi_path)
  mask = io.imread(mask_path)
  mask = Image.fromarray(mask)  
  width, height = wsi.dimensions

  batch_size = 32
  for i in range(0, width, tile_size):
    for j in range(0, height, tile_size*batch_size):  
      run_batch(i,j,j+(tile_size*batch_size),folder, tile_size=256)
  


  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

This stage is processing for Training_phase_2_045


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [06:15<00:00, 62.55s/it]
